# Cosa sono veramente le **Neural Networks**? (parte II)

In questa lezione proviamo ad **esplorare alcuni aspetti pratici che riguardano le reti neurali**: come implementarne (i) l'architettura, (ii) il meccanismo di addestramento, e (iii) cioè come usare architetture precedentemente create.

Per farlo, ci serviremo di un esempio di classificazione intuitivo e di un insieme di dati reali.

## Strumenti di lavoro

Oltre al notebook per editare testo e codice, ci serviamo di una libreria per la manipolazione di un nuovo tipo di dato: il tensore.

Inoltre, per questa lezione, ci ispiriamo ai seguenti materiali:
1. [un tutorial ufficiale di `PyTorch`](https://pytorch.org/tutorials/beginner/nn_tutorial.html) firmato da _Jeremy Howard_ ([fast.ai](https://www.fast.ai)), da dove prendiamo in prestito il codice per la creazione di una rete neurale _from scratch_
2. [un tutorial sulle basi di `PyTorch`](https://github.com/yunjey/pytorch-tutorial) dell'utente [@yunjey](https://github.com/yunjey)

## Prerequisiti 

Il tutorial assume la conoscenza dei seguenti concetti basilari: 
- **operazioni tra tensori** (vettori multidimensionali) che sono solitamente oggetto di un corso base di algebra lineare
- qualche nozione di **teoria delle probabilità**
- **un po' di Python** o di un linguaggio ad oggetti similare

# Elementi di `PyTorch`

In [ ]:
import torch # importiamo la libreria PyTorch

Per **manipolare tensori** e **definire le reti neurali** useremo il framework [PyTorch](https://pytorch.org/docs/stable) che mette a disposizione *classi*, *metodi* e *funzioni* che aiutano nella creazione e nell'addestramento delle reti neurali.

I tensori di `PyTorch` hanno una proprietà molto interessate, **ci permettono di tenere traccia delle trasformazioni dell'input** - quelle che applichiamo al `torch.tensor`.
`PyTorch` salva queste operazioni in uno spazio speciale nella struttura dati del `torch.tensor`.

> Quello di cui abbiamo bisogno è un metodo per calcolare rapidamente la derivata della funzione che ha trasformato i nostri dati, assumendo che questa sia derivabile. 

> Vogliamo misurare il tasso di cambiamento della funzione di trasformazione rispetto alla/alle variabile/variabili di input.

Per fare in modo che `PyTorch` tenga traccia delle operazioni di un `torch.tensor` fissiamo il parametro `requires_grad=True`. 

In [ ]:
# creiamo alcuni tensori
x = torch.tensor(1., requires_grad=True) # un tensore di input x = [1.0]
w = torch.tensor(2., requires_grad=True) # un tensore dei pesi w = [2.0]
b = torch.tensor(3., requires_grad=True) # un tensore dei bias b = [3.0]

# definiamo il modello del percettrone
y = w * x + b # y = [5.0]

# calcoliamo i gradienti
y.backward(retain_graph=True)

In [ ]:
y


Per calcolare le derivate parziali $\frac{\partial y }{\partial x}, \frac{\partial y }{\partial w}, \frac{\partial y }{\partial b}$ possiamo "guardare" nell'attributo `grad` del relativo `torch.tensor`.
Certo, questo calcolo lo possiamo fare anche "a mente", giusto?

In [ ]:
print(x.grad)
print(w.grad)
print(b.grad) # ?

## Come è fatta una **Neural Network** in `PyTorch`

Costruiamo una rete neurale completa servendoci dei tipi di `PyTorch`, ci limitiamo ad un semplice modello di regressione lineare.

In [ ]:
n_esempi = 10 # prendiamo 10 esempi
n_variabili = 3 # ogni esempio è rappresentato da 3 variabili (larghezza, altezza, ampiezza)
inputs = torch.randn(n_esempi, n_variabili) 
targets = torch.randn(n_esempi, 1) # dataset di train

In [ ]:
print(inputs[0], targets[0])

In [ ]:
neuroni_di_ingresso = 3
neuroni_di_uscita = 1
linear = torch.nn.Linear(neuroni_di_ingresso, neuroni_di_uscita) # il percettrone in una linea di codice

In [ ]:
linear.weight # il tensore dei pesi embedded nel modello

In [ ]:
linear.bias # il tensore dei bias embedded nel modello

In [ ]:
outputs = linear(inputs) # otteniamo le predizioni

In [ ]:
# guardiamo ad ogni target e relativa predizione
for i in range(len(outputs)): 
  print(targets[i].data, outputs[i].data)

### Come misuriamo la qualità della nostra predizione? 

Abbiamo bisogno di una quantità che ci dica come il nostro modello approssima il "vero" comportamento dei dati - ovvero, il _criterion_ del modello lineare.

Scegliamo un _criterion_ semplice: il **Mean Squared Error** (MSE), che calcola la media delle distanze tra i dati di train e le predizioni. 

$$
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat y_i)^2
$$

dove:
- $N$ è il numero di `inputs`
- $y_i$ è il valore `target` per l'`input` $i$-esimo
- $\hat y_i$ è il valore `output` per l'`input` $i$-esimo

In [ ]:
def MSE(N, y, hat_y):
  mse = 0
  for i in range(N):
    mse += (y[i] - hat_y[i])**2
  return 1/N * mse

Il calcolo della _loss_ può essere fatto con la nostra funzione `MSE` ...

In [ ]:
loss = MSE(n_esempi, outputs, targets)
loss

... oppure con la funzione che ci fornisce `PyTorch` ...

In [ ]:
criterion = torch.nn.MSELoss()
loss = criterion(outputs, targets) # chiamo la funzione di calcolo della loss
loss

Per effettuare l'aggiornamento dei pesi e dei bias calcoliamo i gradienti dei tensori con il metodo `backward()`

In [ ]:
loss.backward()

In [ ]:
linear.weight.grad # il gradiente dei pesi

In [ ]:
linear.bias.grad # il gradiente dei bias

Aggiorniamo il tensore dei pesi sottraendo il gradiente appena calcolato, ricordiamo di tenere in conto anche un oppportuno valore per il `learning rate` (diciamo $0.01$).

$$
w_i = w_i - ( \frac{\partial L}{\partial w} * lr ) \qquad \forall w_i \in w
$$

In [ ]:
w = linear.weight.data
print(w)
w_grad = linear.weight.grad.data
for i in range(len(w)):
  w[i] -= w_grad[i] * 0.01
print(w)

Lo stesso calcolo va fatto per il tensore dei bias.

$$
b_i = b_i - ( \frac{\partial L}{\partial b} * lr ) \qquad \forall b_i \in b
$$

In [ ]:
b = linear.bias.data
print(b)
b_grad = linear.bias.grad.data
for i in range(len(b)):
  b[i] -= b_grad[i] * 0.01
print(b)

**Oppure** possiamo usare (come nel caso del _criterion_) la funzione `SGD` messa a disposizione da `PyTorch`.

In [ ]:
print(linear.weight.data)
print(linear.bias.data)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)
optimizer.step()
print(linear.weight.data)
print(linear.bias.data)

In alternativa possiamo definire un `optimizer`, un tipo di `PyTorch` che esegue esattamente lo stesso comportamento appena implementato.

> I tensori di pesi e bias prendono - insieme - il nome di **parametri della rete**. 
Sono tutti quei valori il cui aggiornamento comporta l'**apprendimento** del modello.

Nel gergo delle reti esiste il "numero di parametri apprendibili della rete" = somma delle dimensioni del _tensore_ di **pesi** e del _tensore_ di **bias**.

In [ ]:
sum([p.numel() for p in linear.parameters()])

# [MNIST](https://en.wikipedia.org/wiki/MNIST_database) $-$ _handwritten digits classification_

In [ ]:
import torchvision

![MNIST example](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Il problema che vogliamo risolvere è classificare correttamente il valore rappresentato nella fotografia di una cifra scritta a mano ([clicca qui se vuoi saperne di più](http://yann.lecun.com/exdb/mnist/)).

Ma prima pensiamo ad una alternativa, come potremmo rappresentare questo problema con un sistema di _decision rules_?

> *Riusciamo a trovare una congiunzione/disgiunzione di relazioni tra attributi e valori per la prima fotografia in alto a sinistra nell'immagine? Come la  distinguiamo da quella in basso a destra?*

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='sample_data',
                                           train=True, 
                                           transform=torchvision.transforms.ToTensor(),  
                                           download=True)

val_dataset = torchvision.datasets.MNIST(root='sample_data', 
                                          train=False, 
                                          transform=torchvision.transforms.ToTensor())

In [ ]:
train_dataset

In [ ]:
val_dataset

Diamo un occhio ad un file di esempio per vedere come è rappresentato l'input.

In [ ]:
scegli_un_numero = 214124 % len(train_dataset)
immagine, classe = train_dataset[scegli_un_numero] # il dataset è fatto di tuple (immagine, classe)

size = (28,28) # decidiamo come vogliamo "vedere" l'input

import matplotlib.pyplot as plt
plt.imshow(immagine.reshape(size)) #, cmap="gray")
plt.show()

# print(immagine)
print(classe)

## Tutto insieme in un esempio con `PyTorch`

Inizializziamo gli iperparametri per l'addestramento, potrebbe essere possibili provare di verse combinazioni per questi valori prima di incontrare la configurazione migliore.

In [ ]:
learning_rate = 1e-4 # potenze negative del 10 descrescente
batch_size = 64 # il numero di input che vengono processati dal modello (progessione in potenze del 2 crescenti)

In [ ]:
n_pixels = 28*28 
n_variabili = n_pixels # ?
n_classi = 10 # ?

In [ ]:
model = torch.nn.Linear(n_variabili, n_classi)
sum([p.numel() for p in model.parameters()])

In [ ]:
# confrontiamo con il nostro calcolo del numero di parametri 
28 * 28 * 10 + 10 #larghezza * altezza * n_classi + bias

In [ ]:
# modello alternativo un po' più complesso
model = torch.nn.Sequential(
    torch.nn.Linear(n_variabili,n_variabili//2),
    torch.nn.ReLU(),
    torch.nn.Linear(n_variabili//2, n_classi)
)
sum([p.numel() for p in model.parameters()])

In [ ]:
model

Il _criterion_ che scegliamo è il **Negative Log Likelihood** (NLL), utile quando le classi del nostro problema sono **valori** (al plurale) discreti.
Si tratta di una funzione di costo che ci dice quanto "male" il modello sta funzionando: 

> _the lower, the better_.

**Ma** NLL ha bisogno di un input espresso come il logaritmo di una distribuzione di probabilità (la _log probability_, di cui calcolare la _negative likelihood_. 

Per saperne qualcosa di più potete leggere [questo post su Medium](https://medium.com/deeplearningmadeeasy/negative-log-likelihood-6bd79b55d8b6) e [la pagina relativa alla Cross entropy di Wikipedia](https://en.wikipedia.org/wiki/Cross_entropy).

Per trasformare l'input in una distribuzione di probabilità ci serviamo della funzione `log softmax`:

$$
f(x_{i}) = \log\left(\frac{{\rm e}^{x_i}}{ \sum_j {\rm e}^{x_j}}\right)
$$

In [ ]:
log_softmax = torch.nn.LogSoftmax(dim=1)
criterion = torch.nn.NLLLoss() # Negative Log Likelihood  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

max_epoch = 10
total_step = len(train_loader)
for epoch in range(max_epoch):
  for i, (immagini, classi) in enumerate(train_loader):
    inputs = immagini.reshape(-1, n_variabili)

    outputs = model(inputs)
    log_probs = log_softmax(outputs)
    loss = criterion(log_probs, classi)
    
    optimizer.zero_grad() # resetto l'aggiornamento
    loss.backward() # calcolo i gradienti
    optimizer.step() # aggiorno i parametri
        
    if (i+1) % 100 == 0:
      print(f'Epoca [{epoch+1}/{max_epoch}], Step [{i+1}/{total_step}], Loss: {loss:.4f}')

Testiamo su un insieme di dati mai visti durante il training.

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

corrette = 0
totali = 0
for immagini, classi in val_loader:
  inputs = immagini.reshape(-1, n_variabili)
  outputs = model(inputs)
  _, pred = torch.max(outputs.data, 1)
  totali += classi.size(0)
  corrette += (pred == classi).sum()

print(f'Accuracy of the model on 10000 validation images: {100 * corrette // totali} %')

# Grazie per la vostra attenzione :-)

Mi potete contattare alla mail istituzionali [stefanopio \[dot\] zingaro \[at\] unibo \[dot\] it](mailto:stefanopio.zingaro@unibo.it)

[Attribuzione Internazionale Creative Commons 4.0](http://creativecommons.org/licenses/by-sa/4.0/).